<a href="https://colab.research.google.com/github/jbpost2/ST-554-Big-Data-with-Python/blob/main/02_Common_Streaming_Tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Common Streaming Tasks

> Justin Post

Let's get familiar with some of the common tasks we need to do with streaming data!

Note: These types of webpages are built from Jupyter notebooks (`.ipynb` files). You can access your own versions of them by [clicking here](https://colab.research.google.com/github/jbpost2/ST-554-Big-Data-with-Python/blob/main/01_Programming_in_python/20-Plotting_pandas.ipynb). **It is highly recommended that you go through and run the notebooks yourself, modifying and rerunning things where you'd like!**

## Common Issues: Preprocessing, Detecting Trends, Counting, and Averages

When we work with streaming data, the data pretty much always comes with some kind of time stamp or date-type data. That means we should first do a recap of how to deal with time data.

Then we'll discuss the basics of processing data and sending an alert or writing to a log.

As we often need to combine two streams, we'll recap the basic ideas of joining two data frames.

Finally, we take on bascis statistical summaries of streaming data such as
- Counting events
- Increasing and decreasing trends
- Means and standard deviations
- Summary stats over time windows


## Air Quality Data

Let's start with some data we can use to emulate some streaming concepts

From: <https://archive.ics.uci.edu/ml/datasets/Air+quality>

>  ...dataset contains 9358 instances of hourly averaged responses from an array of 5 metal oxide chemical sensors embedded in an Air Quality Chemical Multisensor Device

We'll read this data in. Note that the data has a `Date` and a `Time` column

In [3]:
import pandas as pd
air_data = pd.read_csv("https://www4.stat.ncsu.edu/online/datasets/AirQualityUCI.csv", sep = ";", decimal = ",")
air_data

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10/03/2004,18.00.00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,NaN,NaN
1,10/03/2004,19.00.00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,NaN,NaN
2,10/03/2004,20.00.00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,NaN,NaN
3,10/03/2004,21.00.00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,NaN,NaN
4,10/03/2004,22.00.00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The `Date` and `Time` columns can't be handled easily. For instance, there is no way to determine the 'day' easily.

In [4]:
air_data.Date.day #no attribute for day!

AttributeError: 'Series' object has no attribute 'day'

## Dates and Times in Python

Most standard date/time operations can be handled via the `datetime` module.  This module includes the following data types:

- `date`: attributes of `year`, `month`, `day`
- `time`: attributes of `hour`, `minute`, `second`, `microsecond`, and `tzinfo`
- `datetime`: attributes of both `date` and `time`
- `timedelta`: difference between two `date`, `time` or `datetime` instances

With this functionality we can add and subtract dates/times to get meaningful info while keeping the data in a more readable format (rather than say looking at the data as days since Jan 1, 1960)!


### Dates and Times in `pandas`

Rather than using the `datetime`'s functionality, we'll focus on how `pandas` handles date data via `NumPy`'s functionality.

- `pandas` uses `NumPy`'s `datetime64` and `timedelta64` dtypes

These have [very similar functionality](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html) for doing useful things with dates!

We saw that our columns that were supposed to be dates were read in as strings. We can coerce a string to a date/time variable.

Date time variables have information on both the date and the time of day such as `"04-01-2022 10:00"`. Of course we need to carefully specify whether this is April or January as different countries do this differently!

In [5]:
a = pd.to_datetime(["04-01-2022 10:00"], dayfirst=True)
a

DatetimeIndex(['2022-01-04 10:00:00'], dtype='datetime64[ns]', freq=None)

As mentioned, date-time type variables have useful attributes we can pull from them.

In [6]:
a.day

Index([4], dtype='int32')

In [7]:
a.hour

Index([10], dtype='int32')

There are also useful methods to get descriptive names.

In [8]:
a.day_name()

Index(['Tuesday'], dtype='object')

In [9]:
a.month_name()

Index(['January'], dtype='object')

As mentioned, now that we have a date we can actually subtract two dates to get useful information (while still seeing the date in a nice format).

In [10]:
b = pd.to_datetime(["04-01-2022 11:00"])
a-b

TimedeltaIndex(['-88 days +23:00:00'], dtype='timedelta64[ns]', freq=None)

With our data, we can try to read in the columns appropriately with options on the `pd.read_csv()` function.

In [11]:
air_data = pd.read_csv("https://www4.stat.ncsu.edu/online/datasets/AirQualityUCI.csv",
                       sep = ";",
                       decimal = ",",
                       parse_dates = [["Date", "Time"]])
air_data.info()

<ipython-input-11-4ae5e19be4ba>:1: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  air_data = pd.read_csv("https://www4.stat.ncsu.edu/online/datasets/AirQualityUCI.csv",


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9471 entries, 0 to 9470
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date_Time      9471 non-null   object 
 1   CO(GT)         9357 non-null   float64
 2   PT08.S1(CO)    9357 non-null   float64
 3   NMHC(GT)       9357 non-null   float64
 4   C6H6(GT)       9357 non-null   float64
 5   PT08.S2(NMHC)  9357 non-null   float64
 6   NOx(GT)        9357 non-null   float64
 7   PT08.S3(NOx)   9357 non-null   float64
 8   NO2(GT)        9357 non-null   float64
 9   PT08.S4(NO2)   9357 non-null   float64
 10  PT08.S5(O3)    9357 non-null   float64
 11  T              9357 non-null   float64
 12  RH             9357 non-null   float64
 13  AH             9357 non-null   float64
 14  Unnamed: 15    0 non-null      float64
 15  Unnamed: 16    0 non-null      float64
dtypes: float64(15), object(1)
memory usage: 1.2+ MB


<ipython-input-11-4ae5e19be4ba>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  air_data = pd.read_csv("https://www4.stat.ncsu.edu/online/datasets/AirQualityUCI.csv",


This read those two columns in as a single column of the appropriate type! We now have all the nice functionality to deal with dates.

Lastly, let's rename the `CO(GT)` column since we'll use that one in the upcoming tasks.

In [12]:
air_data = air_data.rename(columns = {'CO(GT)': 'co_gt'})

## Preprocessing & Sending alerts

As data comes in we often need to:

- Check if the data is missing
- Check if data is in an appropriate range
- etc.

If whatever condition is not met we then want to print an alert, write the event to a file, send an email, etc.

Later we'll jump to pyspark and talk about how to handle actual streaming data. For now, let's emulate having data streaming in using a simple for loop. We can think of each iteration of the loop as the next data value coming in.

Using our `air_data` object, let's focus on the `co_gt` variable (true hourly averaged CO concentration (mg/m^3))

- 'Take data in over time' (via a loop over the rows)
- If the data exceeds 8 we print a message

In [14]:
for i in range(air_data.shape[0]):
    if air_data.iloc[i].co_gt > 8:
        print("High CO Concentration at " + str(air_data.Date_Time[i]) + ", loop", str(i))

High CO Concentration at 15/03/2004 09.00.00, loop 111
High CO Concentration at 22/10/2004 18.00.00, loop 5424
High CO Concentration at 25/10/2004 18.00.00, loop 5496
High CO Concentration at 26/10/2004 17.00.00, loop 5519
High CO Concentration at 26/10/2004 18.00.00, loop 5520
High CO Concentration at 02/11/2004 20.00.00, loop 5690
High CO Concentration at 04/11/2004 18.00.00, loop 5736
High CO Concentration at 05/11/2004 17.00.00, loop 5759
High CO Concentration at 17/11/2004 18.00.00, loop 6048
High CO Concentration at 19/11/2004 19.00.00, loop 6097
High CO Concentration at 19/11/2004 20.00.00, loop 6098
High CO Concentration at 23/11/2004 18.00.00, loop 6192
High CO Concentration at 23/11/2004 19.00.00, loop 6193
High CO Concentration at 23/11/2004 20.00.00, loop 6194
High CO Concentration at 23/11/2004 21.00.00, loop 6195
High CO Concentration at 24/11/2004 20.00.00, loop 6218
High CO Concentration at 26/11/2004 18.00.00, loop 6264
High CO Concentration at 26/11/2004 21.00.00, loo

Reasonably straight forward! Of course we might have some other tasks that we'd check for. We can use if/else logic for that.

- 'Take data in over time' (via a loop over the rows)
- If the data exceeds 8 we print a message
- If the data is less than 0 we print a message (-200 represents missing here)

Write either occurrence to a log file (or perhaps a database) rather than printing to the console. Using the menus on the left, create a directory called `logs` to run the code below.

In [18]:
for i in range(air_data.shape[0]):
    temp = air_data.iloc[i]
    dt = temp.Date_Time
    value = temp.co_gt
    if value > 8:
        with open('logs/COHigh.txt', 'a') as f:
            f.write(str(dt) + ", " + str(value) + "\n")
    elif value < 0:
        with open('logs/COInvalid.txt', 'a') as f:
            f.write(str(dt) + ", " + str(value) + "\n")

We may also want to check if something is 'down'. This can mean a lot of things but if something is down, we might want to be notified. We can use python to send an email!

- Unfortunately, it seems that doing this with gmail isn't doable at this point.
- I'll follow [this article](https://www.mailersend.com/blog/send-email-with-python#sending-emails-with-smtplib)!

First we need to have the `dotenv` package. I'll install this vai `pip`.

In [21]:
!pip install dotenv

Now let's read in some module things.

In [22]:
import os
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from dotenv import load_dotenv

Let's set up things to send the email. I've deleted the password here so you'll need to create your own account and what-not to run this code!

In [24]:
# Load environment variables from .env file
load_dotenv()

# Set up email details from environment variables
sender_email = "MS_uW2WFe@trial-eqvygm0z9k8l0p7w.mlsender.net"
password = "removed"
smtp_server = "smtp.mailersend.net"
smtp_port = "587"

# Define the receiver email
receiver_email = "st554testemail@gmail.com"

subject = "Sensor down!"
#we'll set up the body of the email in the loop

Now we'll simulate data coming in via our loop. I'm going to restrict the number of rows to cycle through to 1000 as I only want a couple of emails sent (there is a limited number of free ones on the account).

If we observe six missing observations in a row then we'll send an email as a warning that something seems to be down. To do so:

- We'll initiate a `missing` variable at 0
- If the value is `-200` then we'll add one to missing
- If the value is not `-200` then we reset `missing` to 0
- If the value of `missing` becomes 6 then we send an email

In [29]:
#initate the value
missing = 0

#loop through the first 1000 ros
for i in range(1000):
    #grab the row of interest
    temp = air_data.iloc[i]
    #get the date time and co values
    dt = temp.Date_Time
    value = temp.co_gt

    #Check if the value is large, add to log, reset missing
    if value > 8:
        with open('logs/COHigh.txt', 'a') as f:
            f.write(str(dt) + ", " + str(value) + "\n")
        missing = 0
    #else, check if it is missing (<0), add to log, increase missing
    elif value < 0:
        with open('logs/COInvalid.txt', 'a') as f:
            f.write(str(dt) + ", " + str(value) + "\n")
        if value == -200:
            missing += 1
        #if missing is 6, send the email!
        if missing == 6:
            body = "We have an issue at " + str(dt) + " to resolve!"
            # Create the email message
            message = MIMEMultipart()
            message["From"] = sender_email
            message["To"] = receiver_email
            message["Subject"] = subject
            # Attach the email body to the message
            message.attach(MIMEText(body, "plain"))
            # Establish a connection to the SMTP server and send the email
            try:
                # Connect to the SMTP server
                with smtplib.SMTP(smtp_server, smtp_port) as server:
                    server.starttls()  # Secure the connection
                    server.login(sender_email, password)  # Log in to the SMTP server
                    server.sendmail(
                        sender_email, receiver_email, message.as_string()
                    )  # Send the email
                print("Email sent successfully")
            except Exception as e:
                print(f"Error: {e}")
    #if a reasonable value set missing to 0
    else:
        missing = 0

Email sent successfully
Email sent successfully
Email sent successfully


Checking my email - it worked!
![](https://www4.stat.ncsu.edu/online/datasets/email.jpg)

## Combining Data Streams

- Often have multiple data streams that need to be combined

    + Usually combined via a shared key or time stamps

- Once combined we can then preprocess/summarize/etc.

[Example](https://docs.databricks.com/_static/notebooks/stream-stream-joins-python.html) of streams using google ads type data.

![](https://www4.stat.ncsu.edu/online/datasets/impressions_clicks.png)

To understand the important things for this example we need to define two terms:

- Impression - ad seen by a user
- Clicks - ad was clicked on by user

We may get data each time we get an impression and each time we get a click. There will be more impressions than clicks of course.

Usually there is a userID of some kind that we would want to join this data on. But essentially, once we have a unique 'key' or at least criteria for combining the streams the ideas are the same as SQL type joins.

Just as an example, let's simulate some fake impressions and clicks data with unique userIDs (there would likely be other information in each data stream that would come along as well).


In [32]:
import pandas as pd
import numpy as np
np.random.seed(10)
impressions = pd.DataFrame({
  'userId': range(500),
  'impressionTime': (pd.to_datetime('2022-01-01') + pd.to_timedelta(np.random.rand(500), unit = "D")).sort_values()
})
impressions.head()

,userId,impressionTime
0,0,2022-01-01 00:02:32.033682620
1,1,2022-01-01 00:05:41.130210730
2,2,2022-01-01 00:12:28.161050454
3,3,2022-01-01 00:13:16.703184279
4,4,2022-01-01 00:14:04.126023142


In [46]:
clicks = impressions.iloc[np.random.randint(size = 30, low = 0, high = 499)].sort_index()
#add a slightly positive time delta to get the clicktime
clicks['clickTime'] = clicks.impressionTime + pd.to_timedelta(np.random.rand(30)/100, unit = "D")
clicks.drop(columns = 'impressionTime', axis = 1, inplace = True)
clicks.head()

,userId,clickTime
22,22,2022-01-01 00:59:13.878226348
45,45,2022-01-01 02:11:55.596574979
62,62,2022-01-01 02:48:28.205289126
99,99,2022-01-01 04:36:43.967578509
111,111,2022-01-01 05:10:26.852991101


We can "join" these too with the usual SQL join idea. Here we just use `pd.merge()`. We'll use `clicks` as our right table and do a 'right' join to only return rows that match the `clicks` rows.

In [50]:
combined = pd.merge(left = impressions, right = clicks, on = "userId", how = 'right')
combined.head()

,userId,impressionTime,clickTime
0,22,2022-01-01 00:55:08.550121653,2022-01-01 00:59:13.878226348
1,45,2022-01-01 02:05:14.156931085,2022-01-01 02:11:55.596574979
2,62,2022-01-01 02:44:06.170676406,2022-01-01 02:48:28.205289126
3,99,2022-01-01 04:25:00.608628852,2022-01-01 04:36:43.967578509
4,111,2022-01-01 04:59:24.901278622,2022-01-01 05:10:26.852991101


Now we could find the time it took someone to click (of those that clicked).

In [51]:
(combined.clickTime-combined.impressionTime).head()

,0
0,0 days 00:04:05.328104695
1,0 days 00:06:41.439643894
2,0 days 00:04:22.034612720
3,0 days 00:11:43.358949657
4,0 days 00:11:01.951712479


Idea is easy :)  Will be harder with actual data streams!

# Recap

Often need to check/validate the data

- Basic checks to create an issue log file or entries in an issues data base

- Might send email as a notification

- Could of course do basic ETL type operations as the data comes in too!
    + Filter observations
    + Quick transformations
    + Combine data streams
    + etc.

# Finding Averages & Standard Deviations

We all know how to find averages and standard deviations.  Given data $y_1,...,y_n$

$$\bar{y} = \frac{\sum_{i=1}^{n}y_i}{n}$$
$$s = \sqrt{\frac{1}{n-1}\sum_{i=1}^{n}(y_i-\bar{y})^2} = \sqrt{\frac{1}{n-1}\left(\sum_{i=1}^{n}y_i^2-\frac{(\sum_{i=1}^{n}y_i)^2}{n}\right)}$$

---

## Quick Video
This video shows an example of using pandas for plotting. Remember to pop the video out into the full player.

The notebook written in the video is [available here](https://jbpost2.github.io/ST-554-Big-Data-with-Python/01_Programming_in_python/Learning_Python.html).

In [ ]:
from IPython.display import IFrame
IFrame(src="https://ncsu.hosted.panopto.com/Panopto/Pages/Embed.aspx?id=f4e2bc16-1757-4f1e-8df9-b103016c97d2&autoplay=false&offerviewer=true&showtitle=true&showbrand=true&captions=false&interactivity=all", height="405", width="720")

---

# Recap

- Creating visualizations is an important part of an EDA

- Goal: Describe the distribution

- `pandas` has nice functionality for creating common plots

    + `.plot()` method

- We only covered a few plots but the concepts are similar for others

- May want to [check out seaborn for quick ways to do fancier plots!](https://seaborn.pydata.org/tutorial/introduction.html)

If you are on the course website, use the table of contents on the left or the arrows at the bottom of this page to navigate to the next learning material!

If you are on Google Colab, head back to our course website for [our next lesson](https://jbpost2.github.io/ST-554-Big-Data-with-Python/01_Programming_in_python/21-Error_Handling.html)!